In [18]:
#ライブラリをインポートする
%matplotlib inline 
import numpy as np
import pandas as pd 
import seaborn as sns
import datetime
from matplotlib import pyplot as plt 
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import log_loss,accuracy_score,confusion_matrix,precision_recall_fscore_support


In [19]:
#データを読み込む
main_data=pd.read_csv("./4104_16930_compressed_ks-projects-201801.csv/ks-projects-201801.csv")

In [20]:
#データの内容を確認する(欠損値の確認、カテゴリデータの確認）
#15変数中8変数がカテゴリデータ＆"name"と"used_pledged_realに欠損値を確認
main_data.head(5)
main_data.info()
main_data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 15 columns):
ID                  378661 non-null int64
name                378657 non-null object
category            378661 non-null object
main_category       378661 non-null object
currency            378661 non-null object
deadline            378661 non-null object
goal                378661 non-null float64
launched            378661 non-null object
pledged             378661 non-null float64
state               378661 non-null object
backers             378661 non-null int64
country             378661 non-null object
usd pledged         374864 non-null float64
usd_pledged_real    378661 non-null float64
usd_goal_real       378661 non-null float64
dtypes: float64(5), int64(2), object(8)
memory usage: 43.3+ MB


ID                     0
name                   4
category               0
main_category          0
currency               0
deadline               0
goal                   0
launched               0
pledged                0
state                  0
backers                0
country                0
usd pledged         3797
usd_pledged_real       0
usd_goal_real          0
dtype: int64

In [21]:
#とりあえず欠損値がある行は削除して、stateが成功か失敗だけにして、2値dataに変換をかけておく
main_data=main_data.dropna(how="any")
main_data.isnull().sum()

#stateが成功か失敗だけにして
main_data["state"].value_counts()
main_data1=main_data.query("state=='failed'")
main_data2=main_data.query("state=='successful'")
main_data=pd.concat([main_data1,main_data2])
print(main_data.head(5))
main_data["state"].value_counts()

#2値dataに変換をかけておく
main_data.loc[main_data["state"]=='successful',"state"]=1
main_data.loc[main_data["state"]=='failed',"state"]=0
main_data["state"].value_counts()

           ID                                               name  \
0  1000002330                    The Songs of Adelaide & Abullah   
1  1000003930      Greeting From Earth: ZGAC Arts Capsule For ET   
2  1000004038                                     Where is Hank?   
3  1000007540  ToshiCapital Rekordz Needs Help to Complete Album   
7  1000030581  Chaser Strips. Our Strips make Shots their B*tch!   

         category main_category currency    deadline     goal  \
0          Poetry    Publishing      GBP  2015-10-09   1000.0   
1  Narrative Film  Film & Video      USD  2017-11-01  30000.0   
2  Narrative Film  Film & Video      USD  2013-02-26  45000.0   
3           Music         Music      USD  2012-04-16   5000.0   
7          Drinks          Food      USD  2016-03-17  25000.0   

              launched  pledged   state  backers country  usd pledged  \
0  2015-08-11 12:12:28      0.0  failed        0      GB          0.0   
1  2017-09-02 04:43:57   2421.0  failed       15      

0    197611
1    133851
Name: state, dtype: int64

In [22]:
#データの予測が正しいか見るためにデータの整形＆確認(カテゴリdataは数字に置き換える)
#LabelEncoderのインスタンスを生成
le = LabelEncoder()

#対象とするカラムのリストを作成しておく
category_culumns=["category","main_category","currency","country"]

for i in category_culumns:
    print(i)
    le=le.fit(main_data[i])
    main_data[i]=le.transform(main_data[i])


category
main_category
currency
country


In [23]:
"""
〇used_XXXX:この変数なんや？→どうやらUSDでexchangeした金額らしい。制約金額よりも目標金額のほうが影響しそう。
　　　　　　USD以外の金額で絞って確認してみるか・・・。
            なんかusd_pledged_real>usd_goal_realでも失敗が混じってたりするな
            まあ。いいかデータも少ないし・・・4件とやし
"""
#print(main_data.loc[main_data["currency"]!=13,["state","usd pledged","usd_pledged_real","usd_goal_real"]])

main_data_temp=main_data.query("currency!=13 & usd_pledged_real>usd_goal_real")[["state","usd pledged","usd_pledged_real","usd_goal_real"]]

print(main_data_temp["state"].value_counts())
print(main_data_temp.loc[main_data_temp["state"]==0,["state","usd pledged","usd_pledged_real","usd_goal_real"]])

main_data_temp=main_data.query("currency!=13 & usd_pledged_real<usd_goal_real")[["state","usd pledged","usd_pledged_real","usd_goal_real"]]

print(main_data_temp["state"].value_counts())
print(main_data_temp.loc[main_data_temp["state"]==1,["state","usd pledged","usd_pledged_real","usd_goal_real"]])

main_data_temp=main_data.query("currency==13 & usd_pledged_real<usd_goal_real")[["state","usd pledged","usd_pledged_real","usd_goal_real"]]

print(main_data_temp["state"].value_counts())
print(main_data_temp.loc[main_data_temp["state"]==1,["state","usd pledged","usd_pledged_real","usd_goal_real"]])

main_data_temp=main_data.query("currency==13 & usd_pledged_real>usd_goal_real")[["state","usd pledged","usd_pledged_real","usd_goal_real"]]

print(main_data_temp["state"].value_counts())
print(main_data_temp.loc[main_data_temp["state"]==0,["state","usd pledged","usd_pledged_real","usd_goal_real"]])


1    23759
Name: state, dtype: int64
Empty DataFrame
Columns: [state, usd pledged, usd_pledged_real, usd_goal_real]
Index: []
0    45552
1        1
Name: state, dtype: int64
        state  usd pledged  usd_pledged_real  usd_goal_real
335859      1     27778.24          27765.07       27884.98
0    152053
1         4
Name: state, dtype: int64
        state  usd pledged  usd_pledged_real  usd_goal_real
72483       1      3599.69           3599.69         3600.0
151490      1     30716.00          30716.00        36000.0
232422      1      7999.65           7999.65         8000.0
275777      1      1999.66           1999.66         2000.0
1    106035
0         4
Name: state, dtype: int64
        state  usd pledged  usd_pledged_real  usd_goal_real
16780       0     23020.01          23020.01        23000.0
58558       0      3087.50           3087.50         3000.0
216545      0     10797.00          10797.00        10000.0
298960      0      6183.00           6183.00         6000.0


In [24]:
"""
〇name:ただのプロジェクト名で成功確率に関係なさそう→意外と名前のながさとか影響しそう
"""
#名前の長さをとる列を作り出す
main_data["name_length"]=main_data["name"].str.len()

print(main_data[["name","name_length"]])

                                                     name  name_length
0                         The Songs of Adelaide & Abullah           31
1           Greeting From Earth: ZGAC Arts Capsule For ET           45
2                                          Where is Hank?           14
3       ToshiCapital Rekordz Needs Help to Complete Album           49
7       Chaser Strips. Our Strips make Shots their B*tch!           49
...                                                   ...          ...
378642  ÉPOUVANTAILS : 28mm Figurines de jeux pour KIN...           55
378644                               The Manual Bar Blade           20
378646                                The Dog Coffee Book           19
378651                                       AT THE BEACH           12
378652                        Beach Wrestling Documentary           27

[331462 rows x 2 columns]


In [25]:
"""
〇launchedとdeadline:この差分、要するに応募期間が長いほうが成功しやすいとかはありそう（差分を変数にする）
  
"""

main_data["deadline"]=pd.to_datetime(main_data["deadline"])
main_data["launched"]=pd.to_datetime(main_data["launched"])
date_delta=main_data["deadline"]-main_data["launched"]
#main_data["period"]=main_data["period"].days
main_data["period"]=date_delta.dt.days
print(main_data[["period","deadline","launched"]])
print(type(main_data["period"][0]))

        period   deadline            launched
0           58 2015-10-09 2015-08-11 12:12:28
1           59 2017-11-01 2017-09-02 04:43:57
2           44 2013-02-26 2013-01-12 00:20:50
3           29 2012-04-16 2012-03-17 03:24:11
7           44 2016-03-17 2016-02-01 20:05:12
...        ...        ...                 ...
378642      26 2017-10-31 2017-10-04 11:26:44
378644      21 2015-12-15 2015-11-23 07:33:14
378646      42 2013-11-30 2013-10-18 21:35:04
378651      29 2014-03-22 2014-02-20 01:00:16
378652      29 2015-04-28 2015-03-29 21:30:33

[331462 rows x 3 columns]
<class 'numpy.int64'>


In [26]:
#データの最終確認(結局期間は関係なさそう)
display(main_data)
main_data.describe()
main_data.info()
main_data=main_data.drop(["name","period"],axis=1)
main_data=main_data.drop(["ID","goal","launched","deadline","pledged","usd pledged","usd_pledged_real"],axis=1)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,name_length,period
0,1000002330,The Songs of Adelaide & Abullah,108,12,5,2015-10-09,1000.0,2015-08-11 12:12:28,0.00,0,0,9,0.00,0.00,1533.95,31,58
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,93,6,13,2017-11-01,30000.0,2017-09-02 04:43:57,2421.00,0,15,21,100.00,2421.00,30000.00,45,59
2,1000004038,Where is Hank?,93,6,13,2013-02-26,45000.0,2013-01-12 00:20:50,220.00,0,3,21,220.00,220.00,45000.00,14,44
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,90,10,13,2012-04-16,5000.0,2012-03-17 03:24:11,1.00,0,1,21,1.00,1.00,5000.00,49,29
7,1000030581,Chaser Strips. Our Strips make Shots their B*tch!,41,7,13,2016-03-17,25000.0,2016-02-01 20:05:12,453.00,0,40,21,453.00,453.00,25000.00,49,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378642,999929142,ÉPOUVANTAILS : 28mm Figurines de jeux pour KIN...,136,8,4,2017-10-31,1000.0,2017-10-04 11:26:44,1246.00,1,35,8,66.72,1452.47,1165.70,55,26
378644,999934908,The Manual Bar Blade,113,4,13,2015-12-15,3500.0,2015-11-23 07:33:14,6169.00,1,120,21,6169.00,6169.00,3500.00,20,21
378646,999943841,The Dog Coffee Book,19,12,13,2013-11-30,950.0,2013-10-18 21:35:04,1732.02,1,31,21,1732.02,1732.02,950.00,19,42
378651,999969812,AT THE BEACH,23,10,1,2014-03-22,5000.0,2014-02-20 01:00:16,5501.00,1,78,3,5019.92,4983.69,4529.81,12,29


<class 'pandas.core.frame.DataFrame'>
Int64Index: 331462 entries, 0 to 378652
Data columns (total 17 columns):
ID                  331462 non-null int64
name                331462 non-null object
category            331462 non-null int32
main_category       331462 non-null int32
currency            331462 non-null int32
deadline            331462 non-null datetime64[ns]
goal                331462 non-null float64
launched            331462 non-null datetime64[ns]
pledged             331462 non-null float64
state               331462 non-null int64
backers             331462 non-null int64
country             331462 non-null int32
usd pledged         331462 non-null float64
usd_pledged_real    331462 non-null float64
usd_goal_real       331462 non-null float64
name_length         331462 non-null int64
period              331462 non-null int64
dtypes: datetime64[ns](2), float64(5), int32(4), int64(5), object(1)
memory usage: 50.5+ MB


In [27]:
#ロジスティック回帰を行う
y=main_data["state"].values
X=main_data.drop("state",axis=1).values

#max_iter=学習の反復の最大回数
clf=SGDClassifier(loss="log",penalty='none',max_iter=10000,fit_intercept=True,random_state=1234,tol=1e-3)
clf.fit(X,y)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log',
              max_iter=10000, n_iter_no_change=5, n_jobs=None, penalty='none',
              power_t=0.5, random_state=1234, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [28]:
#モデルの評価
y_pred=clf.predict(X)
print(f"対数尤度:{log_loss(y,y_pred):.3f}")
print(f"正答率:{accuracy_score(y,y_pred)*100:.3f}")


対数尤度:2.786
正答率:91.933


In [31]:
#モデルを評価する（RECALL,ACCURACY,PRECISION)
# Precision, Recall, F1-scoreを計算
precision, recall, f1_score, _ = precision_recall_fscore_support(y, y_pred)

# カテゴリ「2000万以上」に関するPrecision, Recall, F1-scoreを表示
print(f'適合率（Precision） = {100 * precision[0]:.3f}%')
print(f'再現率（Recall） = {100 * recall[0]:.3f}%')
print(f'F1値（F1-score） = {100 * f1_score[0]:.3f}%')

適合率（Precision） = 94.039%
再現率（Recall） = 92.322%
F1値（F1-score） = 93.172%


In [33]:
#クロス表を作る
conf_mat=pd.DataFrame(confusion_matrix(y,y_pred),index=['実際：×','実際：〇'],columns=['予測:×','予測:〇'])
conf_mat

,予測:×,予測:〇
実際：×,182438,15173
実際：〇,11565,122286
